In [1]:
import pandas as pd
import numpy as np


In [2]:


# Function to create a matrix of random likelihood scores
def create_likely(N, M):
    """
    Create a matrix of random likelihood scores.

    Parameters:
    - N (int): Number of agents.
    - M (int): Number of items.

    Returns:
    - DataFrame: DataFrame representing the likelihood scores.
    """
    matrix_random = np.random.randn(N, M)
    rows = ['a' + str(i) for i in range(1, N + 1)]
    columns = ['b' + str(i) for i in range(1, M + 1)]
    df = pd.DataFrame(matrix_random, index=rows, columns=columns)
    return df



# Function to create the network matrix
def create_network(length, k):
    """
    Create a network matrix representing connections between agents.

    Parameters:
    - length (int): Number of agents.
    - k (int): Number of neighbors each agent connects to on each side.

    Returns:
    - DataFrame: DataFrame representing the network matrix.
    """
    matrix = np.zeros((length, length), dtype=int)

    for i in range(length):
        for j in range(1, k + 1):  # Each node has k neighbors on each side
            matrix[i, (i + j) % length] = 1  # Right neighbor
            matrix[i, (i - j) % length] = 1  # Left neighbor
        matrix[i, i] = 0  # Remove self-connection

    # Convert the matrix to a DataFrame
    df = pd.DataFrame(matrix)

    # Rename rows and columns
    df.columns = [f'a{i+1}' for i in range(length)]
    df.index = [f'a{i+1}' for i in range(length)]

    return df





In [4]:
import pandas as pd
import os

N = 10  # Number of agents
M = 10   # Number of items
gamma_list = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]  # List of gamma values
time = int(M/5)  # Time steps
k = 2  # Number of neighbors

for gamma in gamma_list:
    for x in range(15):  # 15 repetitions of the experiment
        likely = create_likely(N,M)  # Create likely matrix
        network = create_network(N,k)  # Create network of agents
        items = list(likely.columns)  # Get the list of items
        df_purchase = pd.DataFrame()  # Initialize the purchase matrix
        chosen_items = {f'a{i+1}': set() for i in range(N)}  # Create sets to store chosen items for each agent

        # At time step 0, each agent picks the most likely item
        for n in range(N):
            df_purchase.loc['t=0', 'a' + str(n + 1)] = likely.loc['a' + str(n + 1)].idxmax()
            chosen_items['a' + str(n + 1)].add(likely.loc['a' + str(n + 1)].idxmax())

        # Iterate through time steps
        for n in range(time):
            for m in range(N):
                # Create the agent's network
                agent_network = []
                for column_name, value in network["a" + str(m+1)].items():
                    if value == 1:
                        agent_network.append(column_name)
                
                # Determine the items chosen by neighboring agents
                dictionary = {f'b{i}': 0 for i in range(1, M + 1)}
                for b in range(n + 1):
                    for a in df_purchase[agent_network].loc['t=' + str(b)]:
                        if a in dictionary:
                            dictionary[a] += 1 / len(df_purchase[agent_network].columns)

                # Calculate opinion for every item
                scaled_dict = {k: ((v * gamma) + (1 - gamma) * (likely.loc['a' + str(m + 1), str(k)])) for k, v in dictionary.items()}

                # Remove already chosen items
                for item in chosen_items['a' + str(m + 1)]:
                    if item in scaled_dict:
                        del scaled_dict[item]

                # Select the item with the highest value
                max_value = max(scaled_dict.values())
                max_keys = [k for k, v in scaled_dict.items() if v == max_value]
                chosen_item = np.random.choice(max_keys)
                df_purchase.loc['t=' + str(n+1), 'a' + str(m + 1)] = chosen_item

                # Add the chosen item to the set of chosen items for this agent
                chosen_items['a' + str(m + 1)].add(chosen_item)
        
        # Create the purchase matrix with 0s and 1s
        df_purchase_with_0_1 = pd.DataFrame(np.zeros((N, M), dtype=int),
                        index=[f'a{i+1}' for i in range(N)],
                        columns=[f'b{i+1}' for i in range(M)])
        for n in range(N):
            for item in df_purchase['a' + str(n + 1)]:
                if item in chosen_items['a' + str(n+1)]:
                    df_purchase_with_0_1.loc['a'+str(n+1), item] = 1
        
        # Create the directory path for the file
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        directory_path = os.path.join(desktop_path, f'N={N} -- M={M} -- k={k} -- gamma={gamma} -- t={time}')
        os.makedirs(directory_path, exist_ok=True)

        # Define the file path
        file_path = os.path.join(directory_path, f'{x}run.xlsx')

        # Write the Excel file
        with pd.ExcelWriter(file_path) as writer:
            likely.to_excel(writer, sheet_name='Likely Matrix', index=True)
            df_purchase.to_excel(writer, sheet_name='Purchase Matrix', index=True)
            df_purchase_with_0_1.to_excel(writer, sheet_name='Purchase Matrix(0,1)', index=True)

        # Print likely and df_purchase variables (commented out)
        # print(likely)
        # print(df_purchase)
        # chosen_items
